### What-If Tool image smile detection

In this demo we demonstrate the use of what-if-tool for image recognition models. Our task is to predict if a person is smiling or not. We operate on a subset of celeba dataset.

Copyright 2019 Google LLC.
SPDX-License-Identifier: Apache-2.0


In [0]:
#@title Install latest TensorFlow and the What-If Tool widget if running in colab {display-mode: "form"}

# If running in colab then pip install, otherwise no need.
try:
  import google.colab
  !pip install --upgrade witwidget
except Exception:
  pass

In [0]:
#@title Download the pretrained keras model files and sample images

!curl -L https://storage.googleapis.com/what-if-tool-resources/smile-demo/smile-male-50000.hdf5 -o ./smile-model.hdf5
!curl -L https://storage.googleapis.com/what-if-tool-resources/smile-demo/test_subset.zip -o ./test_subset.zip

In [0]:
!unzip test_subset.zip

In [0]:
import pandas as pd

data = pd.read_csv('celeba/data_test_subset.csv')

In [0]:
#@title Define helper functions for dataset conversion from csv to tf.Examples
import numpy as np
import tensorflow as tf
import os
from PIL import Image
from io import BytesIO


# Converts a dataframe into a list of tf.Example protos.
def df_to_examples(df, columns=None, images_path=''):
  examples = []
  if columns == None:
    columns = df.columns.values.tolist()
  for index, row in df.iterrows():
    example = tf.train.Example()
    for col in columns:
      if df[col].dtype is np.dtype(np.int64):
        example.features.feature[col].int64_list.value.append(int(row[col]))
      elif df[col].dtype is np.dtype(np.float64):
        example.features.feature[col].float_list.value.append(row[col])
      elif row[col] == row[col]:
        example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
    if images_path:
      fname = row['image_id']
      with open(os.path.join(images_path, fname), 'rb') as f:
        im = Image.open(f)
        buf = BytesIO()
        im.save(buf, format= 'PNG')
        im_bytes = buf.getvalue()
        example.features.feature['image/encoded'].bytes_list.value.append(im_bytes)
    examples.append(example)
  return examples

# Converts a dataframe column into a column of 0's and 1's based on the provided test.
# Used to force label columns to be numeric for binary classification using a TF estimator.
def make_label_column_numeric(df, label_column, test):
  df[label_column] = np.where(test(df[label_column]), 1, 0)

In [0]:
examples = df_to_examples(data, images_path='celeba/img_test_subset_resized/')

In [0]:
#@title Load the keras models
from tensorflow.keras.models import load_model

model1 = load_model('smile-model.hdf5')

In [0]:
# WIT predict functions:
def custom_predict(examples_to_infer):
  def load_byte_img(im_bytes):
    buf = BytesIO(im_bytes)
    return np.array(Image.open(buf), dtype=np.float64) / 255.

  ims = [load_byte_img(ex.features.feature['image/encoded'].bytes_list.value[0]) for ex in examples]
  preds = model1.predict(np.array(ims))
  return preds

In [0]:
#@title Invoke What-If Tool for the data and two models {display-mode: "form"}
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

num_datapoints = 250  #@param {type: "number"}
tool_height_in_px = 700  #@param {type: "number"}

# Setup the tool with the test examples and the trained classifier
config_builder = WitConfigBuilder(examples[:num_datapoints])
config_builder.set_custom_predict_fn(custom_predict)
wv = WitWidget(config_builder, height=tool_height_in_px)

#### Exploration ideas

